# Random Forest

Decision Tree는 Overfitting 가능성이 높다는 약점이 있다. Random Forest는 여러개의 Decision Tree를 형성하고 동일한 데이터로부터 복원 추출하여 Decision Tree에 통과 시킨다. 그 후, 각 Tree가 분류한 결과를 취합하는 앙상블(Ensenble) 방식의 모델이다. 이렇게 많은 수의 Tree를 생성함으로써 일부 Tree에 Overfitting이 발생하더라도 Overfitting이 결과에 영향을 주는 것을 방지한다.

----

**TIP)**   
*Ensemble*   

<img src='./image/bagging_boosting.jpg' width=500 height=400>

- Bagging (Bootstrap Aggregation)    
Bootstraping(복원 추출하여 train set을 만드는 것, 비복원 추출은 Pasting이라 함) 된 샘플을을 모델에 학습시킨다. 그 후, 학습된 모델의 결과를 집계하여 투표(Voting) 방식으로 최종 결과 값을 도출하며 연속적인 데이터에 대해서는 평균으로 집계한다. 이러한 Bagging은 Overfitting을 피하고 분산(Variance)을 낮추는 효과를 가진다. 대표적으로, Random Forest 모델이 있다.    

- Boosting    
Boosting은 성능 자체의 강화와 어려운 문제의 해결에 초점을 맞추고 있으며 편향(bias)가 높은 경우 이를 해결하는 효과가 있다. Bagging 처럼 복원 샘플링 방식을 사용하지만 순차적으로 학습이 진행되며 결과에 따라 오답에 대해 높은 가중치 정답에 대해 낮은 가중치를 부여해 가며 오답에 집중하는 방식으로 학습이 이루어진다. 높은 정확도를 가지지만 Outlier에는 취약것이 단점이다. 대표적으로 AdaBoost, XGBoost, GradientBoost 등이 있다.     

<img src='./image/stacking_cv.png' width=600 height=400>

- Stacking    
Stacking은 서로 상이한 개별의 모델들이 예측한 데이터를 다시 meta data set으로 사용해서 학습한다. 즉, 개별 모델들(Base Learner)과 최종모델(Meta Learner)이 별도로 존재한다. 원본 데이터를 Base와 Meta가 함께 사용하므로 Overfitting에 취약한 경우가 많은데, 이를 해결하기 위해 Cross Validation 방식을 사용하는 경우가 많다.

----